In [2]:
# import libraries related to querying links and downloading files from the web
from datetime import datetime
from IPython.display import display, clear_output, Video
import importlib
import ipywidgets as widgets
import os
import pandas as pd
import pipmag as pm


In [102]:
# reload the pipmag module to make sure that the latest version is used
importlib.reload(pm)
print('Imported updated pipmag.py')

Imported updated pipmag.py


### ⚙️ Generating the dataframe from SST quicklooks

In [15]:
# Print the years for which the La Palma Observatory has data at UiO
obs_years = pm.get_obs_years()

In [16]:
# Get the observing dates for all the years
obs_dates = pm.get_obs_dates(obs_years)
obs_dates_list = pm.get_obs_dates_list(obs_dates)
# print the first, last and total number of observing dates
print(f'first entry: {obs_dates_list[0]}\nlast entry : {obs_dates_list[-1]}\ntotal observing dates: {len(obs_dates_list)}')

first entry: 2020-09-19
last entry : 2015-09-09
total observing dates: 109


In [28]:
# check if all_media_links.pkl exists then load the pickle file, otherwise get the links
if not os.path.isfile('all_media_links.pkl'):
    video_links = pm.get_video_liks(obs_dates) # get the video links, one for each observing date
    image_links = pm.get_image_links(obs_dates) # get the image links, one for each observing date
    all_image_links = pm.get_all_links(image_links) # get all the image links, one for each image
    all_video_links = pm.get_all_links(video_links) # get all the video links, one for each video
    # print the number of video and image links and all the video and image links 
    print(f'number of video links: {len(all_video_links)}\nnumber of image links: {len(all_image_links)}')
    print(f'video links: {len(all_video_links)}\nimage links: {len(all_image_links)}')
    all_media_links = all_image_links + all_video_links # combine the image and video links
    all_media_links = sorted(all_media_links) #sort the list of links
    # print the total number of media links
    print(f'total number of media links: {len(all_media_links)}')
    # save all the media links as a pickle file
    pm.save_pickle(all_media_links, 'all_media_links.pkl')
else:
    # load the pickle file
    all_media_links = pm.load_pickle('all_media_links.pkl')
    print(f'total number of media links: {len(all_media_links)}')

loaded all_media_links.pkl successfully
total number of media links: 6861


In [29]:
# get the date and time from the links and find the links that do not have date and time and save them as a list
date_time_from_all_media_links, date_time_not_found = pm.get_date_time_from_link_list(all_media_links)
# remove all the links that do not have a date and time from all_media_links
all_media_links_with_date_time = [link for link in all_media_links if link not in date_time_not_found]
# print the number of links that contain date and time and the number of links that do not contain date and time
print(f'number of links with date and time: {len(all_media_links_with_date_time)}\nnumber of links without date and time: {len(date_time_not_found)}')
invalid_date = pm.get_invalid_dates(date_time_from_all_media_links)
# remove the entries from date_time_from_all_media_links that are not in the correct format
# date_time_from_all_media_links = [date for date in date_time_from_all_media_links if date not in invalid_dates]

number of links with date and time: 6817
number of links without date and time: 44
All dates in date_time_list are valid


In [33]:
# convert the date and time to datetime format
date_time_from_all_media_links_datetime = pm.convert_to_datetime(date_time_from_all_media_links)
# get the unique date_time_from_all_media_links_datetime  values
unique_date_time_from_all_media_links_datetime = list(set(date_time_from_all_media_links_datetime))
# print the number of unique date_time_from_all_media_links_datetime values
print(f'number of unique date_time_from_all_media_links_datetime values: {len(unique_date_time_from_all_media_links_datetime)}')

number of unique date_time_from_all_media_links_datetime values: 749


In [34]:
# create a dataframe with the date_time_from_all_media_links_datetime as the index and the all_media_links as the column
df = pd.DataFrame(all_media_links_with_date_time, index=date_time_from_all_media_links_datetime, columns=['links'])
#print first, last and total number of entries in the dataframe
print(f'first entry: {df.index[0]}\nlast entry : {df.index[-1]}\ntotal entries: {len(df.index)}')

first entry: 2013-06-30 09:15:50
last entry : 2022-09-08 08:40:11
total entries: 6817


In [35]:
# group the dataframe by the time index and combine the links into a list
df = df.groupby(df.index).agg({'links': lambda x: list(x)})
# print the first, last and total number of entries in the dataframe
print(f'first entry: {df.index[0]}\nlast entry : {df.index[-1]}\ntotal entries: {len(df.index)}')

first entry: 2013-06-30 09:15:50
last entry : 2022-09-17 14:34:13
total entries: 749


In [36]:
# add a column called 'obs_id' and set it equal to the row number of the dataframe
# add the 'id' column
df['obs_id'] = range(1, len(df) + 1)
# set the index as 'obs_id' and add a column for the date and time
df['date_time'] = df.index
df = df.set_index('obs_id')
# add a column for the number of links in each row
df['num_links'] = df['links'].apply(lambda x: len(x))
# add columns for the year, month and day to the dataframe
df['year'] = df['date_time'].apply(lambda x: x.year)
df['month'] = df['date_time'].apply(lambda x: x.month)
df['day'] = df['date_time'].apply(lambda x: x.day)
# add a column for the time of day
df['time'] = df['date_time'].apply(lambda x: x.time())
# add a column called 'target' and set it equal to None
df['target'] = None
instrument_keywords={'CRISP': ['wb_6563','ha','Crisp-R'],'CROMIS':['Chromis'],'IRIS':['sji']}
# apply the get_instrument_info function to the 'links' column of the dataframe and add the result to a new column called 'instruments'
df['instruments'] = df['links'].apply(lambda x: pm.get_instrument_info(x, instrument_keywords))
# apply the get_links_with_string function to the 'links' column of the dataframe with the strings 'mp4' and 'mov' and add the result to a new column called 'video_links'
df['video_links'] = df['links'].apply(lambda x: pm.get_links_with_string(x, ['mp4','mov']))
# apply the get_links_with_string function to the 'links' column of the dataframe with the strings 'jpg' and 'png' and add the result to a new column called 'image_links'
df['image_links'] = df['links'].apply(lambda x: pm.get_links_with_string(x, ['jpg','png']))
#pm.get_links_with_string(df.iloc[0]['links'], ['mp4','mov'])
# make the columns date-time, year, month, day, time, instruments, target, video_links, image_links, links, num_links
df = df[['date_time', 'year', 'month', 'day', 'time', 'instruments', 'target', 'video_links', 'image_links', 'links', 'num_links']]
# print a summary of the dataframe
df.info()
# save the dataframe as a pickle file
df.to_pickle('la_palma_obs_data.pkl')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 749 entries, 1 to 749
Data columns (total 11 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   date_time    749 non-null    datetime64[ns]
 1   year         749 non-null    int64         
 2   month        749 non-null    int64         
 3   day          749 non-null    int64         
 4   time         749 non-null    object        
 5   instruments  534 non-null    object        
 6   target       0 non-null      object        
 7   video_links  749 non-null    object        
 8   image_links  749 non-null    object        
 9   links        749 non-null    object        
 10  num_links    749 non-null    int64         
dtypes: datetime64[ns](1), int64(4), object(6)
memory usage: 70.2+ KB


### ➡️ Start here : Load the existing dataframe

In [ ]:
df = pd.read_pickle('la_palma_obs_data.pkl')
df.info()

In [106]:
selector = pm.VideoSelector(df)
selector.create_widget()
multi_updater = pm.DataUpdater(df, ['target', 'instruments'])
multi_updater.display()

Dropdown(description='Year:', options=(2013, 2014, 2015, 2016, 2017, 2018, 2019, 2020, 2021, 2022), value=2013…

Dropdown(description='Month:', options=(), value=None)

Dropdown(description='Day:', options=(), value=None)

Dropdown(description='Time:', options=(), value=None)

Dropdown(description='Links:', options=(), value=None)

...

Text(value='', description='Index:')

Text(value='', description='target:')

Text(value='', description='instruments:')

Button(description='Update', style=ButtonStyle())

Output()